# Care Centers in Castellon – Analysis Template

This notebook demonstrates how to use your modularized code for data loading, cleaning, mapping, and analysis.

In [ ]:
# --- 1. Notebook Setup: Import paths, libraries, and modules ---
import sys
sys.path.append('../src')  # Ensure the src folder is in the path

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from data import fetch_osm_data, clean_data
from mapping import generate_castellon_map
from visualization import display_images
import config

## 2. Data Collection

Use the data module to fetch OSM data for Castellón.

In [ ]:
city_name = "Castellón, Valencian Community, Spain"
gdf_nodes, gdf_edges = fetch_osm_data(city_name)
print(f"Nodes: {gdf_nodes.shape}, Edges: {gdf_edges.shape}")

## 3. Data Inspection

Preview the raw dataframes.

In [ ]:
gdf_edges.head()

## 4. Data Cleaning

Clean the edge dataframe using the clean_data utility.

In [ ]:
gdf_edges_clean = clean_data(gdf_edges)
gdf_edges_clean.head()

## 5. Quick Data Summary

Check the number of columns and rows after cleaning.

In [ ]:
print(f"Cleaned Edges shape: {gdf_edges_clean.shape}")
print(f"Columns: {gdf_edges_clean.columns.tolist()}")

## 6. Interactive Map of Castellon

Use the mapping module to generate a folium map.

In [ ]:
castellon_map = generate_castellon_map()
castellon_map

## 7. Display Local Images (Optional)

If you have images in your data/images directory, display them using the visualization module.

In [ ]:
image_paths = [
    (config.IMAGES_PATH + "img1_El-Fadri.jpg", "Fig. 1: El Fadrí bell tower"),
    (config.IMAGES_PATH + "img2_Planetarium.jpeg", "Fig. 2: Planetarium"),
    (config.IMAGES_PATH + "img3_lush_Parque_ribalta.jpeg", "Fig. 3: Parque Ribalta"),
]
display_images(image_paths)

## 8. Further Analysis

- Continue with spatial analysis, accessibility studies, statistical summaries, or visualizations as needed.
- Import or implement more functions in `src/` as your project grows.